In [1]:
import pandas as pd
import pickle
import numpy as np
import re
import gc
from sklearn import preprocessing

In [2]:
df = pd.read_csv('../gen_data/train2.csv')
#train0 = pd.read_csv('../original_data/sales_train.csv.gz')
#df = pd.read_csv('../gen_data/train1.csv')

x_train_list = pickle.load(open('../gen_data/x_train--features3.ipynb--.pickle','rb'))
x_test_list = pickle.load(open('../gen_data/x_test--features3.ipynb--.pickle','rb'))

# BUILD A DF RELATING ITEM CAT ID AND SHOP ID TO PARSED VARS THEN MERGE

# maybe drop duplicates before all this

In [3]:
df = df.drop_duplicates()

In [4]:
#Parse City data
cities_encoder = preprocessing.LabelEncoder()
cities_series = pd.Series([re.search('(.*?) ',n).group() for n in df.shop_name],index=df.index)
cities_encoder.fit(cities_series)
df['city'] = cities_encoder.transform(cities_series)
del cities_series

In [5]:
#Parse item category data
item_type_encoder = preprocessing.LabelEncoder()
item_type_series = df.item_category_name.map(lambda x: re.split('-',x)[0])
item_type_encoder.fit(item_type_series)
df['item_type'] = item_type_encoder.transform(item_type_series)
del item_type_series

In [6]:
#parse mor item_category data
item_info_encoder = preprocessing.LabelEncoder()
item_info_series = df.item_category_name.map(lambda x: '-'.join(re.split('-',x)[1:]) if len(re.split('-',x))>1 else 'NAN')
item_info_encoder.fit(item_info_series)
df['item_info'] = item_info_encoder.transform(item_info_series)
del item_info_series

In [7]:
df = df[['shop_id','item_id','item_category_id','city','item_type','item_info']]

In [8]:
#function to downcast data types to 32 bits
def downcast(df):
    float_cols = [col for col in df if df[col].dtype=='float64']
    int_cols = [col for col in df if df[col].dtype=='int64']

    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int32)
    
    return(df)

In [9]:
df = downcast(df)
gc.collect()

7

# exp begin

In [10]:
x_train_exp = x_train_list[0][(x_train_list[0].date_block_num.isin([30,31])) & (x_train_list[0].shop_id==2) & (x_train_list[0].item_id==33)]

In [11]:
x_train_exp

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,sum_shop_sales_back_12,item_cnt_month_back_12,sum_item_cat_sales_back_12,sum_item_cat_shop_sales_back_12,month,numdays,mean_prevmonth_item_price,sd_prevmonth_item_price,mean_prevmonth_shop_item_price,sd_above_market
5524842,30,2,33,37.0,0.0,0.0,0.0,0.0,0.0,12.0,...,0.0,0.0,0.0,0.0,jul,31,199.0,0.000000,NaN,0.000000
5754486,31,2,33,0.0,15.0,763.0,1.0,3291.0,29.0,0.0,...,0.0,0.0,0.0,0.0,aug,31,194.6,17.041127,199.0,0.258199


In [12]:
df = df.drop_duplicates()

In [13]:
df.shape

(424124, 6)

In [14]:
x_train_exp.merge(df,on=['item_id','item_category_id','shop_id'],how='left')

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,sum_item_cat_shop_sales_back_12,month,numdays,mean_prevmonth_item_price,sd_prevmonth_item_price,mean_prevmonth_shop_item_price,sd_above_market,city,item_type,item_info
0,30,2,33,37.0,0.0,0.0,0.0,0.0,0.0,12.0,...,0.0,jul,31,199.0,0.000000,NaN,0.000000,1.0,11.0,1.0
1,31,2,33,0.0,15.0,763.0,1.0,3291.0,29.0,0.0,...,0.0,aug,31,194.6,17.041127,199.0,0.258199,NaN,NaN,NaN


# exp end

In [15]:
x_train_list[0].shape

(6186922, 70)

In [16]:
x_train_list = list(map(lambda x: x.merge(df,on=['item_id','item_category_id','shop_id'],how='left'),x_train_list))
x_test_list = list(map(lambda x: x.merge(df,on=['item_id','item_category_id','shop_id'],how='left'),x_test_list))

In [17]:
x_train_list[0].shape

(6186922, 73)

In [18]:
pickle.dump(x_train_list,open('../gen_data/x_train--strings.ipynb--.pickle','wb'))
pickle.dump(x_test_list,open('../gen_data/x_test--strings.ipynb--.pickle','wb'))

In [19]:
x_train_list[0]

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,sum_item_cat_shop_sales_back_12,month,numdays,mean_prevmonth_item_price,sd_prevmonth_item_price,mean_prevmonth_shop_item_price,sd_above_market,city,item_type,item_info
0,12,2,32,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,jan,31,134.691573,15.602202,NaN,0.000000,1.0,11.0,6.0
1,12,2,33,37.0,42.0,1255.0,1.0,9906.0,55.0,15.0,...,52.0,jan,31,186.571429,16.449056,199.000,0.755580,1.0,11.0,1.0
2,12,2,99,37.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,jan,31,124.167083,25.367019,NaN,0.000000,1.0,11.0,1.0
3,12,2,482,73.0,88.0,1255.0,1.0,412.0,3.0,85.0,...,7.0,jan,31,2866.022727,435.543311,3300.000,0.996404,1.0,15.0,0.0
4,12,2,485,73.0,111.0,1255.0,1.0,412.0,3.0,0.0,...,0.0,jan,31,250.203390,51.145216,300.000,0.973632,1.0,15.0,0.0
5,12,2,804,49.0,26.0,1255.0,1.0,842.0,7.0,0.0,...,8.0,jan,31,227.115385,20.874165,240.000,0.617252,1.0,12.0,38.0
6,12,2,839,73.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,jan,31,3091.707317,344.029210,NaN,0.000000,1.0,15.0,0.0
7,12,2,1007,67.0,247.0,1255.0,1.0,3576.0,10.0,0.0,...,28.0,jan,31,441.207977,155.515705,599.000,1.014637,1.0,14.0,51.0
8,12,2,1406,28.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,jan,31,758.350758,235.080954,NaN,0.000000,1.0,8.0,33.0
9,12,2,1407,19.0,466.0,1255.0,13.0,14490.0,278.0,6.0,...,0.0,jan,31,734.125609,542.392120,1299.000,1.041450,1.0,5.0,12.0
